🔹 STEP 1: Install dependencies

In [1]:
!pip install -q langchain langchain-community langchain-core faiss-cpu PyMuPDF
!pip install -q langchain-groq

🔹 STEP 2: Setup GROQ API Key





In [2]:
# ✅ Set your GROQ API key (replace with your actual key)
import os
os.environ["GROQ_API_KEY"] = "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"


🔹 STEP 3: Upload PDF File

In [3]:
# ✅ Upload PDF
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

ModuleNotFoundError: No module named 'google'

🔹 STEP 4: Load & Split PDF into Chunks

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

print(f"Total chunks: {len(chunks)}")


Total chunks: 121


🔹 STEP 5: Create Vector Store (FAISS) Using Gemini Embeddings

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)


/tmp/ipython-input-6-2898519074.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔹 STEP 6: Create RAG Chain with GROQ LLM




In [ ]:
# ✅ Load LLM & build RAG chain
from langchain_groq.chat_models import ChatGroq
from langchain.chains import RetrievalQA

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.3,
    api_key=os.environ["GROQ_API_KEY"]
)

retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


🔹 STEP 7: Ask for Explanation

In [ ]:
query = "Explain the problem, methodology, and key takeaways"
response = qa_chain.run(query)

print("📝 Blog Explainer:\n")
print(response)


/tmp/ipython-input-8-912533424.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


📝 Blog Explainer:

Based on the provided context, here's an explanation of the problem, methodology, and key takeaways:

**Problem:**
The problem revolves around morphing attacks, which are a threat to border crossing and ID management scenarios. Morphing attacks involve creating a fake identity document (e.g., passport) by morphing the face of an individual into another person's face. The goal of a morphing attack is to deceive human observers, particularly ID experts and border guards, into believing that the individual presenting the document is the genuine owner.

**Methodology:**
To address this problem, researchers are developing methods to detect morphed faces. The methodology involves submitting these methods to ongoing benchmarks, such as the SOTAMD benchmark at the University of Bologna or the U.S. NIST-FRVT-MORPH benchmark. These benchmarks use a sequestered dataset to evaluate the performance of the submitted methods. The evaluation is typically done by fixing the Attack Pr